<a href="https://colab.research.google.com/github/Jonathanpro/myaiblog/blob/master/_notebooks/2021-07-07-huggingface_multi_class_pytorch_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP -  Multi Class Classification - Inference
> A tutorial for applying inference to a multi class classifier with Huggingface & Pytroch.

- toc: true 
- badges: true
- comments: false
- categories: [jupyter, NLP, GPU]
- image: images/chart-preview.png

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.5MB 30.5MB/s 
     |████████████████████████████████| 901kB 38.7MB/s 
     |████████████████████████████████| 3.3MB 29.5MB/s 


In [ ]:
# Importing the libraries needed
import pandas as pd
import numpy as np
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer

In [ ]:
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip

--2021-07-07 14:56:53--  https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29224203 (28M) [application/x-httpd-php]
Saving to: ‘NewsAggregatorDataset.zip’

NewsAggregatorDatas 100%[===================>]  27.87M  23.2MB/s    in 1.2s    

2021-07-07 14:56:54 (23.2 MB/s) - ‘NewsAggregatorDataset.zip’ saved [29224203/29224203]



In [ ]:
!unzip NewsAggregatorDataset.zip

Archive:  NewsAggregatorDataset.zip
  inflating: 2pageSessions.csv       
   creating: __MACOSX/
  inflating: __MACOSX/._2pageSessions.csv  
  inflating: newsCorpora.csv         
  inflating: __MACOSX/._newsCorpora.csv  
  inflating: readme.txt              
  inflating: __MACOSX/._readme.txt   


In [ ]:
# Import the csv into pandas dataframe and add the headers
df = pd.read_csv('newsCorpora.csv', sep='\t', names=['ID','TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])
# df.head()
# # Removing unwanted columns and only leaving title of news and the category which will be the target
df = df[['TITLE','CATEGORY']]
# df.head()

# # Converting the codes to appropriate categories using a dictionary
my_dict = {
    'e':'Entertainment',
    'b':'Business',
    't':'Science',
    'm':'Health'
}

def update_cat(x):
    return my_dict[x]

df['CATEGORY'] = df['CATEGORY'].apply(lambda x: update_cat(x))

encode_dict = {}

def encode_cat(x):
    if x not in encode_dict.keys():
        encode_dict[x]=len(encode_dict)
    return encode_dict[x]

df['ENCODE_CAT'] = df['CATEGORY'].apply(lambda x: encode_cat(x))
labels = list(df['CATEGORY'].unique())
df_mapping=df[['CATEGORY', 'ENCODE_CAT']].drop_duplicates()
df_mapping = df_mapping.reset_index()
id2label = pd.Series(df_mapping.CATEGORY,index=df_mapping.ENCODE_CAT).to_dict()

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased', id2label=id2label)

In [ ]:
class train_data_class(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        title = str(self.data.TITLE[index])
        title = " ".join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        inputs['label'] = torch.tensor(self.data.ENCODE_CAT[index], dtype=torch.long)
        del inputs['token_type_ids']
        return inputs
        """
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'label': torch.tensor(self.data.ENCODE_CAT[index], dtype=torch.long)
        } 
        """
    def __len__(self):
        return self.len

In [ ]:
# Creating the dataset and dataloader for the neural network
MAX_LEN = 512
train_size = 0.8
train_dataset = df.sample(frac=train_size,random_state=200)
test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


training_set = train_data_class(train_dataset, tokenizer, MAX_LEN)
testing_set = train_data_class(test_dataset, tokenizer, MAX_LEN)

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=1*16,  # batch size per device during training
    per_device_eval_batch_size=1*64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=1000,
) # save_strategy='steps', save_steps=5000 

model = AutoModelForSequenceClassification.from_pretrained("/content/gdrive/MyDrive/MC_03_09_07_05_21.bin", local_files_only=True, num_labels=len(labels), id2label=id2label)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    # compute_metrics=compute_metrics,
    train_dataset=training_set,         # training dataset
    eval_dataset=testing_set             # evaluation dataset
)

In [ ]:
class data_prediction(torch.utils.data.Dataset):
	def __init__(self, encodings):
		self.encodings = encodings

	def __getitem__(self, idx):
		item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
		return item

	def __len__(self):
		return len(self.encodings['input_ids'])

In [ ]:

def predict_text(text, trainer=trainer, tokenizer=tokenizer):
    encodings = tokenizer([text], truncation=True, padding=True)
    dataset = data_prediction(encodings)
    result = trainer.predict(dataset)
    return id2label[np.argmax(result.predictions)]

Sample News headline from Google news South Africa

In [ ]:
predict_text("Relieved' Aspen CEO laments impact of US vaccine factory problems on SA ")

***** Running Prediction *****
  Num examples = 1
  Batch size = 64


'Health'

Go to https://news.google.com/topstories?hl=en-US&gl=US&ceid=US:en and select an english speaking region(e.g. UK, US, New Zealand, AUS, Namibia, etc.)
Check the left categories and copy a newline into the 'predict_text' function